In [2]:
import numpy as np
import pandas as pd
import warnings
from datetime import datetime, timedelta
from matplotlib import pyplot as plt


warnings.filterwarnings('ignore')

In [16]:
df = pd.read_csv('data/calendar_compiled.csv')
df_calendar = df.drop(['Unnamed: 0'], axis =1)

In [8]:
df_listing = pd.read_csv('data/listings.csv')

##  EDA

1. missing check

In [19]:
df_listing.isna().sum()

id                                 0
listing_url                        0
name                               1
experiences_offered                0
host_id                            0
host_name                          0
host_since                         0
host_location                     11
host_about                      1999
host_response_time              1100
host_response_rate              1100
host_acceptance_rate            5711
host_is_superhost                  0
host_neighbourhood               479
host_listings_count                0
host_total_listings_count          0
street                             0
neighbourhood                      0
neighbourhood_cleansed             0
neighbourhood_group_cleansed    5711
city                               7
state                              0
zipcode                           30
market                             9
smart_location                     0
country_code                       0
country                            0
l

In [17]:
df_calendar.groupby()

,listing_id,date,price,unavailable
0,3781,20181119,125.0,0
1,3781,20181120,125.0,0
2,3781,20181121,125.0,0
3,3781,20181122,125.0,0
4,3781,20181123,150.0,0
5,3781,20181124,150.0,0
6,3781,20190117,125.0,1
7,3781,20190118,150.0,1
8,3781,20190119,150.0,1
9,3781,20190120,125.0,1


In [9]:
df_listing

,id,listing_url,name,experiences_offered,host_id,host_name,host_since,host_location,host_about,host_response_time,...,bedrooms,beds,bed_type,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people
0,3781,https://www.airbnb.com/rooms/3781,HARBORSIDE-Walk to subway,none,4804,Frank,2008/12/3,Massachusetts,My wife and I and grown children frequently oc...,within a few hours,...,1.0,1,Real Bed,$125.00,$750.00,"$2,250.00",$0.00,$75.00,2,$0.00
1,6976,https://www.airbnb.com/rooms/6976,Mexican Folk Art Showcase in Boston Neighborhood,none,16701,Phil,2009/5/11,"Boston, Massachusetts, United States","I am a middle-aged, single male with a wide ra...",within an hour,...,1.0,1,Real Bed,$65.00,$395.00,"$1,350.00",$0.00,$0.00,1,$30.00
2,8789,https://www.airbnb.com/rooms/8789,Curved Glass Studio/1bd facing Park,none,26988,Anne,2009/7/22,"Boston, Massachusetts, United States",I manage a number of furnished units around Bo...,within a day,...,1.0,1,Real Bed,$99.00,"$1,200.00","$2,900.00","$1,000.00",$250.00,1,$0.00
3,9273,https://www.airbnb.com/rooms/9273,"Stay at ""HARBORVIEW"" Walk to subway",none,4804,Frank,2008/12/3,Massachusetts,My wife and I and grown children frequently oc...,within a few hours,...,2.0,2,Real Bed,$225.00,"$1,400.00","$3,600.00",NaN,$80.00,4,$20.00
4,10730,https://www.airbnb.com/rooms/10730,Bright 1bed facing Golden Dome,none,26988,Anne,2009/7/22,"Boston, Massachusetts, United States",I manage a number of furnished units around Bo...,within a day,...,1.0,1,Real Bed,$150.00,"$1,200.00","$3,000.00","$1,000.00",$250.00,1,$0.00
5,10811,https://www.airbnb.com/rooms/10811,Back Bay Studio on Marlborough St-,none,38997,Michelle,2009/9/16,US,NaN,within a few hours,...,0.0,2,Real Bed,$179.00,NaN,"$2,250.00",$500.00,$100.00,1,$0.00
6,10813,https://www.airbnb.com/rooms/10813,Back Bay Apt Studio-,none,38997,Michelle,2009/9/16,US,NaN,within a few hours,...,0.0,2,Real Bed,$179.00,$800.00,"$2,250.00",$500.00,$100.00,1,$0.00
7,10986,https://www.airbnb.com/rooms/10986,Waterfront Furnished Apt,none,38997,Michelle,2009/9/16,US,NaN,within a few hours,...,0.0,1,Real Bed,$125.00,NaN,NaN,$500.00,$145.00,1,$0.00
8,18711,https://www.airbnb.com/rooms/18711,The Dorset | 3BR 1BA | 1st Floor,none,71783,Lance,2010/1/19,"Boston, Massachusetts, United States","Lance R. Swank, Principal\r\n\r\nHis units off...",within a few hours,...,3.0,3,Real Bed,$126.00,"$1,375.00","$3,500.00",$100.00,$175.00,3,$13.00
9,22354,https://www.airbnb.com/rooms/22354,COPLEY SQ...19TH C.TWNHSE SUNNY RM,none,85770,Robert,2010/2/26,"Boston, Massachusetts, United States",I am a semi retired college professor still te...,within an hour,...,1.0,1,Real Bed,$148.00,NaN,NaN,$0.00,$21.00,1,$15.00
